In [ ]:
#from langchain.chat_models import init_chat_model
#model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

#from langchain_core.messages import HumanMessage
#model.invoke([HumanMessage(content="Hi! I love mangoes")])
#model.invoke([HumanMessage(content="What fruit do I like?")])

# 🧰 Step 1: Install Ana’s Tools

In [ ]:
!pip install langchain-core langgraph>0.2.27

from dotenv import load_dotenv
load_dotenv()

# 🔑 Load your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 🤖 Step 2: Say Hello to Ana

In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 🧠 Load the OpenAI model (GPT-3.5 or GPT-4)
llm = ChatOpenAI(model="gpt-3.5-turbo")  # 💬 Chat-capable model

# 💌 Send a message to the model
response = llm.invoke([HumanMessage(content="Hi! I love mangoes")])  # 🥭

# 📤 Print the AI's response
print(response.content)


Hi! Mangoes are delicious, I agree! Do you have a favorite way to enjoy them?


In [28]:
response = llm.invoke([HumanMessage(content="What fruit do I like?")])
print(response.content)

I'm not sure, can you tell me what your favorite fruit is?


# 🧠 Step 3: Give Ana a Memory with LangGraph

In [38]:
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.checkpoint.memory import MemorySaver

# 🧠 Define your model step (Ana's thinking process)
def call_model(state):
    return {"messages": llm.invoke(state["messages"])}

# 🕸️ Build the LangGraph workflow
workflow = StateGraph(state_schema=MessagesState)

# 🧩 Add a node named 'model1' for the LLM call
workflow.add_node("model1", call_model)

# 🔁 Define the flow: start → model1
workflow.add_edge(START, "model1")

# 💾 Enable memory so Ana remembers past messages
memory = MemorySaver()
ana = workflow.compile(checkpointer=memory)


In [ ]:
# 🧵 Define a unique user thread to track conversation history
config = {"configurable": {"thread_id": "user1"}}

In [39]:
from langchain_core.messages import HumanMessage

# 🗨️ First message: user tells Ana they love mangoes
ana.invoke({"messages": [HumanMessage("Hi Ana, I love mangoes!")]}, config)

# 🧠 Second message: Ana should remember your preference!
ana.invoke({"messages": [HumanMessage("What fruit do I like?")]}, config)


{'messages': [HumanMessage(content='Hi Ana, I love mangoes!', additional_kwargs={}, response_metadata={}, id='815dcee9-41a1-4405-a9bc-5b0e549553e1'),
  AIMessage(content="Hi there! That's great to hear, mangoes are definitely a delicious fruit. Do you have a favorite way to enjoy them?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 15, 'total_tokens': 42, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BnWQkifzOzW9i743KvmTToa5pXfJG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--e697771c-bdce-4fd5-a023-dab7c40b9754-0', usage_metadata={'input_tokens': 15, 'output_tokens': 27, 'total_tokens': 42, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

# 🧙 Step 4: Add a Personality

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 🍳 Define Ana's personality: a friendly cooking assistant!
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly cooking assistant. Always say something about cooking"),
    MessagesPlaceholder(variable_name="messages")
])

# 🧠 LLM call with updated prompt
def call_model(state):
    prompt = prompt_template.invoke({"messages": state["messages"]})
    response = llm.invoke(prompt)
    return {"messages": state["messages"] + [response]}

# 🛠️ Build LangGraph workflow
workflow = StateGraph(state_schema=MessagesState)
workflow.add_node("model2", call_model)
workflow.add_edge(START, "model2")

# 💾 Enable memory
memory = MemorySaver()
ana = workflow.compile(checkpointer=memory)

# 🧵 Set user thread for stateful memory
config = {"configurable": {"thread_id": "user2"}}

# 💬 Test Ana's personality
ana.invoke({"messages": [HumanMessage("Hi Ana! How are you?")]}, config)

{'messages': [HumanMessage(content='Hi Ana! How are you?', additional_kwargs={}, response_metadata={}, id='63661139-63e0-495e-8003-b4bd018892f3'),
  AIMessage(content="Hello! I'm here and ready to help with anything cooking-related. What would you like to know or talk about today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 30, 'total_tokens': 55, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BnWUWuNj2FHrOm9xcAxuyee6kxMEf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--5a0f5a80-e197-4c60-b787-07d546208e5a-0', usage_metadata={'input_tokens': 30, 'output_tokens': 25, 'total_tokens': 55, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

# 🌍 Step 5: Make Ana Speak Any Language

In [44]:
from typing import TypedDict

# 🔑 Load your OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# 🧠 LLM setup (OpenAI model)
llm = ChatOpenAI()

# 🧾 Define Ana’s state with language support
class AnaState(TypedDict):
    messages: list
    language: str

# 🧰 Build prompt template with language conditioning
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Respond in {language}. Be helpful."),
    MessagesPlaceholder(variable_name="messages")
])

# ⚙️ LangGraph node logic
def call_model(state: AnaState):
    prompt = prompt_template.invoke({
        "messages": state["messages"],
        "language": state["language"]
    })
    response = llm.invoke(prompt)
    return {"messages": [response]}

# 🕸️ Build LangGraph workflow
workflow = StateGraph(state_schema=AnaState)
workflow.add_node("model3", call_model)
workflow.add_edge(START, "model3")

# 💾 Enable memory so Ana remembers past messages
memory = MemorySaver()
ana = workflow.compile(checkpointer=memory)

# 🧵 Define user thread (to maintain stateful memory)
config = {"configurable": {"thread_id": "user3"}}

# 💬 Invoke Ana in Hindi
ana.invoke({
    "messages": [HumanMessage("Ana, what's a healthy fruit?")],
    "language": "Hindi"
}, config)


{'messages': [AIMessage(content='सेब एक स्वस्थ फल है। इसमें विटामिन सी, फाइबर और अन्य पोषक तत्व होते हैं जो आपके शरीर के लिए फायदेमंद होते हैं। इसका नियमित सेवन करने से आपकी सेहत में सुधार हो सकता है।', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 26, 'total_tokens': 187, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BnWZcyxqIbtQNn6oIjrH6G6NvWjnZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8f5dbe11-f2b1-401b-8443-88369a1ef2e8-0', usage_metadata={'input_tokens': 26, 'output_tokens': 161, 'total_tokens': 187, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})],
 'language': 'Hindi'}

In [32]:
system_message = "Respond in Hindi. Be helpful."
user_message = "Ana, what's a healthy fruit?"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message}
]

response = llm.invoke(messages)
print(response.content)


स्वागत है। सभी फल स्वास्थ्यवर्धक होते हैं, परन्तु कुछ फल जैसे की सेब, अंगूर, संतरा, आम और पपीता खासतौर से स्वास्थ्य के लिए फायदेमंद होते हैं। ये फल आपकी डाइट में शामिल करना सेहत के लिए लाभकारी हो सकता है।


# 🧹 Step 6: Trim Long Conversations

In [45]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, trim_messages

# 🤖 Setup the LLM (assumed defined earlier)
# llm = ChatOpenAI(model="gpt-3.5-turbo")

# 🧹 1. Setup message trimmer to limit conversation length
trimmer = trim_messages(
    max_tokens=65,
    strategy="last",            # Keep recent messages only
    token_counter=llm,          # Use LLM's tokenizer to count tokens
    include_system=True,        # Always include system messages
    allow_partial=False,        # Do not allow partial messages
    start_on="human",           # Trim starting from last human message
)

# 📝 2. Define prompt template with dynamic language
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Respond in {language}. Be helpful."),
    MessagesPlaceholder(variable_name="messages")
])

# ⚙️ 3. Define model call that trims messages before invoking LLM
def call_model(state: dict):
    trimmed_messages = trimmer.invoke(state["messages"])  # Trim long history
    prompt = prompt_template.invoke({
        "messages": trimmed_messages,
        "language": state["language"]
    })
    response = llm.invoke(prompt)
    return {"messages": [response]}

# 🔗 4. Setup LangGraph workflow
workflow = StateGraph(state_schema=dict)  # Or your AnaState TypedDict
workflow.add_node("model4", call_model)
workflow.add_edge(START, "model4")

# 💾 5. Add memory to persist conversation state
memory = MemorySaver()
ana = workflow.compile(checkpointer=memory)

# 💬 6. Sample conversation messages
messages = [
    SystemMessage(content="You're a good assistant."),
    HumanMessage(content="Hi! I'm Ankita."),
    AIMessage(content="Hi there!"),
    HumanMessage(content="I love data science."),
    AIMessage(content="That's awesome!"),
    HumanMessage(content="Ana, I enjoy working with LangChain."),
    AIMessage(content="LangChain is powerful for building LLM apps."),
    HumanMessage(content="What's a good Python library for data visualization?"),
    AIMessage(content="Matplotlib and Seaborn are great options."),
]

# ❓ 7. Ask question possibly trimmed out from context
config = {"configurable": {"thread_id": "user4"}}
query = "What's my name?"
input_messages = messages + [HumanMessage(query)]
output = ana.invoke({"messages": input_messages, "language": "English"}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't have access to your personal information, so I don't know your name. Is there anything else I can assist you with?


In [34]:
# ❓ 8. Ask question about recent messages (still remembered)
config = {"configurable": {"thread_id": "ana_user_2"}}
query = "Which Python libraries did we discuss?"
input_messages = messages + [HumanMessage(query)]
output = ana.invoke({"messages": input_messages, "language": "English"}, config)
output["messages"][-1].pretty_print()


================================== Ai Message ==================================

We discussed Matplotlib and Seaborn as good options for data visualization libraries in Python.


# ⚡ Step 7: Stream Ana's Responses

In [46]:

# 🚀 Streaming Example: Ana tells a fun fact!
from langchain_core.messages import HumanMessage, AIMessage

# 🧵 Unique thread to track conversation
config = {"configurable": {"thread_id": "ana_stream_user_1"}}

# 💬 User's input question
query = "Hey Ana, can you share a fun science fact?"
language = "English"

# 📨 Add message to input list
input_messages = [HumanMessage(query)]

print("🟢 Streaming Ana's response:")

# 🔄 Stream tokens from Ana as they are generated
for chunk, metadata in ana.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",  # 🧠 Stream by tokens
):
    if isinstance(chunk, AIMessage):  # 🎯 Filter only model responses
        print(chunk.content, end="|")  # ⌨️ Print tokens with separator

print()  # ✅ Newline after complete response

🟢 Streaming Ana's response:
|Of| course|!| Here|'s| a| fun| science| fact|:| Did| you| know| that| honey| never| spo|ils|?| Archae|ologists| have| found| pots| of| honey| in| ancient| Egyptian| tom|bs| that| are| over| |3|,|000| years| old| and| still| perfectly| edible|.| Honey|'s| high| sugar| content| and| acidity| create| an| environment| that| prevents| the| growth| of| bacteria| and| molds|.| It|'s| pretty| amazing|,| isn|'t| it|?||
